In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

def error(msg):
    raise Exception(msg)

def topsis(df, weights, impacts):

    if df.shape[1] < 3:
        error("Input file must contain at least 3 columns.")

    names = df.iloc[:, 0]
    matrix = df.iloc[:, 1:]

    try:
        matrix = matrix.astype(float)
    except:
        error("All columns except first must be numeric.")

    weights = weights.split(",")
    impacts = impacts.split(",")

    if len(weights) != matrix.shape[1]:
        error("Number of weights must match number of columns.")

    if len(impacts) != matrix.shape[1]:
        error("Number of impacts must match number of columns.")

    for i in impacts:
        if i not in ["+", "-"]:
            error("Impacts must be '+' or '-' only.")

    weights = np.array(weights, dtype=float)

    norm = matrix / np.sqrt((matrix**2).sum())
    weighted = norm * weights

    ideal_best = []
    ideal_worst = []

    for i in range(len(impacts)):
        if impacts[i] == "+":
            ideal_best.append(weighted.iloc[:, i].max())
            ideal_worst.append(weighted.iloc[:, i].min())
        else:
            ideal_best.append(weighted.iloc[:, i].min())
            ideal_worst.append(weighted.iloc[:, i].max())

    ideal_best = np.array(ideal_best)
    ideal_worst = np.array(ideal_worst)

    dist_best = np.sqrt(((weighted - ideal_best) ** 2).sum(axis=1))
    dist_worst = np.sqrt(((weighted - ideal_worst) ** 2).sum(axis=1))

    score = dist_worst / (dist_best + dist_worst)
    rank = score.rank(ascending=False)

    df["Topsis Score"] = score
    df["Rank"] = rank

    return df


print("Upload CSV file")
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

weights = input("Enter weights (comma separated): ")
impacts = input("Enter impacts (+ or - comma separated): ")

result = topsis(df, weights, impacts)

output_name = "topsis_result.csv"
result.to_csv(output_name, index=False)

print("\nResult:")
display(result)

files.download(output_name)


Upload CSV file
